1. 小区间质押是否可能获得高额利润

In [2]:
# 获取并存储天级数据
POOL_ID = "0xc31e54c7a869b9fcbecc14363cf510d1c41fa443"

arbitrum_query = """ query get_pools($num_skip: Int, $pool_id: ID!) {
    liquidityPool(id: $pool_id) {
        dailySnapshots(skip: $num_skip, orderBy: day, orderDirection: desc) {
            day
            dailyVolumeUSD
            dailyTotalRevenueUSD
            totalValueLockedUSD
        }
    }
}"""

In [ ]:
num_skip = 0
result = []
is_break = False

In [ ]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


client = Client(
    transport=RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum',
        verify=True,
        retries=5,
    ))

while True:
    print("Querying ticks, num_skip={}".format(num_skip))
    variables = {"num_skip": num_skip, "pool_id": POOL_ID}
    response = client.execute(gql(arbitrum_query), variable_values=variables)
    query_result = response["liquidityPool"]["dailySnapshots"]
    for item in query_result:
        day = int(item["day"])
        # todo: 修改这个部分
        if day <= 19477:
            is_break = True
            break
        result.append(item)

    if is_break == True or len(query_result) == 0:
        break

    num_skip = len(result)

In [4]:
import pandas as pd

from datetime import datetime, timedelta


def unixtime_to_day_before(day_count, unixtime=0):
    current_date = datetime.fromtimestamp(unixtime) + timedelta(days=day_count)
    return current_date.strftime("%Y-%m-%d")


arb_df = pd.DataFrame(result)
arb_df["date"] = arb_df["day"].apply(unixtime_to_day_before)
arb_df.head()

,day,dailyVolumeUSD,dailyTotalRevenueUSD,totalValueLockedUSD,date
0,19556,62645858.87339231038231371303131,31322.92943669615519115685651945,43991968.86397924299999433932886,2023-07-18
1,19555,78836539.34931248039460595756886,39418.2696746562401973029787891,44594966.108651452749034656584,2023-07-17
2,19554,43233649.01354901681799617409771,21616.82450677450840899808705174,46212787.3971710232147209161641,2023-07-16
3,19553,29838778.31630346908133923990871,14919.38915815173454066961995703,46504482.69780273868477842499168,2023-07-15
4,19552,129294121.16599334024223784584379,64647.06058299667012111892292883,46430790.59592811944603207193834,2023-07-14


In [6]:
arb_df["dailyVolumeUSD"] = arb_df["dailyVolumeUSD"].astype(float).round(2)
arb_df["dailyTotalRevenueUSD"] = arb_df["dailyTotalRevenueUSD"].astype(float).round(2)
arb_df["totalValueLockedUSD"] = arb_df["totalValueLockedUSD"].astype(float).round(2)
arb_df["date"] = arb_df["date"].astype(str)
arb_df = arb_df[['date', 'dailyVolumeUSD', 'dailyTotalRevenueUSD', 'totalValueLockedUSD']]

arb_old_df = pd.read_csv("../../data/arb_data_20230718.csv")
# arb_old_df.rename(columns={'dailyTotalVolumeUSD': 'dailyVolumeUSD'}, inplace=True)
arb_df = pd.concat([arb_df, arb_old_df])
arb_df.to_csv("../../data/arb_data_20230718.csv", index=False)

,date,dailyTotalVolumeUSD,dailyTotalRevenueUSD,totalValueLockedUSD
0,2023-04-30,1.294347e+08,64717.36,58969074.13
1,2023-04-29,5.831820e+07,29159.10,59522967.00
2,2023-04-28,1.291540e+08,64576.98,58623166.41
3,2023-04-27,2.670841e+08,133542.06,53907496.47
4,2023-04-26,2.884190e+08,144209.50,50551198.16


In [21]:
# 获取并存储小时级数据
POOL_ID = "0xc31e54c7a869b9fcbecc14363cf510d1c41fa443"

arbitrum_query = """ query get_pools($num_skip: Int, $pool_id: ID!) {
    liquidityPool(id: $pool_id) {
        hourlySnapshots(skip: $num_skip, orderBy: hour, orderDirection: desc) {
            hour
            hourlyVolumeUSD
            hourlyTotalRevenueUSD
            totalValueLockedUSD
        }
    }
}"""

num_skip = 0
result = []
is_break = False

In [24]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


client = Client(
    transport=RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum',
        verify=True,
        retries=5,
    ))

while True:
    print("Querying ticks, num_skip={}".format(num_skip))
    variables = {"num_skip": num_skip, "pool_id": POOL_ID}
    response = client.execute(gql(arbitrum_query), variable_values=variables)
    query_result = response["liquidityPool"]["hourlySnapshots"]
    for item in query_result:
        hour = int(item["hour"])
        # todo: 修改这个部分
        if hour <= 467775:
            is_break = True
            break
        result.append(item)

    if is_break == True or len(query_result) == 0:
        break

    num_skip = len(result)

Querying ticks, num_skip=0
Querying ticks, num_skip=100
Querying ticks, num_skip=200
Querying ticks, num_skip=300
Querying ticks, num_skip=400
Querying ticks, num_skip=500
Querying ticks, num_skip=600
Querying ticks, num_skip=700
Querying ticks, num_skip=800
Querying ticks, num_skip=900
Querying ticks, num_skip=1000
Querying ticks, num_skip=1100
Querying ticks, num_skip=1200
Querying ticks, num_skip=1300
Querying ticks, num_skip=1400
Querying ticks, num_skip=1500
Querying ticks, num_skip=1600


In [27]:
import time

def hour_to_unixtime(hour):
    current_date = datetime.fromtimestamp(0) + timedelta(hours=hour)
    return int(time.mktime(current_date.timetuple()))

In [29]:
arb_hour_df = pd.DataFrame(result)
arb_hour_df["timestamp"] = arb_hour_df["hour"].apply(hour_to_unixtime)
arb_hour_df.head()

,hour,hourlyVolumeUSD,hourlyTotalRevenueUSD,totalValueLockedUSD,timestamp
0,469376,1460878.47258988336766531052831,730.43923629494168383265526443,44233373.3510221773573453678896,1689753600
1,469375,806643.10919596517901233525319,403.3215545979825895061676268,44290156.20582897883020136578084,1689750000
2,469374,1036466.04480128782220493231525,518.23302240064391110246615779,44254886.55727854726053867876903,1689746400
3,469373,656345.72649494087873370659467,328.17286324747043936685329741,44168146.44106156636941494461425,1689742800
4,469372,1637624.49925289785898467064351,818.8122496264489294923353219,44087659.78889045440728093729744,1689739200


In [31]:
arb_hour_df[['timestamp', 'hourlyVolumeUSD', 'hourlyTotalRevenueUSD', 'totalValueLockedUSD']].to_csv("../../data/arb_hour_data_20230719.csv", index=False)

In [13]:
# 比较小时级数据回测质押收益 => 先粗略计算，再根据质押的分布精确计算

import pandas as pd

price_df = pd.read_csv("../../data/eth_usd_polygon_20230718.csv")
price_df.head()

,v,vw,o,c,h,l,t,n
0,9809.27,1942.71,1940.49,1952.54,1958.70,1928.00,1654041600000,30746
1,6089.57,1946.36,1954.20,1946.57,1954.20,1940.00,1654045200000,23571
2,8278.78,1935.83,1946.46,1939.03,1947.35,1928.98,1654048800000,25013
3,7490.52,1931.06,1938.94,1928.44,1940.24,1925.36,1654052400000,21634
4,10282.47,1924.81,1928.38,1932.54,1935.90,1912.03,1654056000000,25834


In [14]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

arb_df = pd.read_csv("../../data/arb_hour_data_20230719.csv")
arb_df.head()

,timestamp,hourlyVolumeUSD,hourlyTotalRevenueUSD,totalValueLockedUSD
0,1689753600,1460878.47,730.44,44233373.35
1,1689750000,806643.11,403.32,44290156.21
2,1689746400,1036466.04,518.23,44254886.56
3,1689742800,656345.73,328.17,44168146.44
4,1689739200,1637624.50,818.81,44087659.79


In [15]:
price_df = price_df[['o', 'c', 'h', 'l', 't']]
price_df["timestamp"] = (price_df['t'] / 1000).astype(int)
price_df.head()

,o,c,h,l,t,timestamp
0,1940.49,1952.54,1958.70,1928.00,1654041600000,1654041600
1,1954.20,1946.57,1954.20,1940.00,1654045200000,1654045200
2,1946.46,1939.03,1947.35,1928.98,1654048800000,1654048800
3,1938.94,1928.44,1940.24,1925.36,1654052400000,1654052400
4,1928.38,1932.54,1935.90,1912.03,1654056000000,1654056000


In [16]:
df = pd.merge(arb_df, price_df, on='timestamp')
df.head()

,timestamp,hourlyVolumeUSD,hourlyTotalRevenueUSD,totalValueLockedUSD,o,c,h,l,t
0,1689721200,1839294.64,919.65,43991968.86,1901.64,1897.72,1903.60,1893.29,1689721200000
1,1689717600,1570102.68,785.05,43971689.85,1897.51,1901.65,1904.20,1896.13,1689717600000
2,1689714000,1699318.65,849.66,44024583.09,1894.62,1897.49,1900.70,1892.85,1689714000000
3,1689710400,2210259.03,1105.13,43916277.35,1892.56,1894.61,1899.80,1889.99,1689710400000
4,1689706800,2770797.59,1385.40,43830655.08,1901.68,1892.67,1905.20,1890.43,1689706800000


In [17]:
df.shape

(1592, 9)

In [18]:
# get all timestamp in arb_df but not in df
from utils import utils

rest_df = arb_df[~arb_df['timestamp'].isin(df['timestamp'])]
rest_df["datetime"] = rest_df["timestamp"].apply(utils.utc_timestamp_to_datetime_str)
rest_df.head(10)

/var/folders/sj/9n8j6m2s6_1g9x78vzd8z0rm0000gn/T/ipykernel_1928/1548438947.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rest_df["datetime"] = rest_df["timestamp"].apply(utils.utc_timestamp_to_datetime_str)


,timestamp,hourlyVolumeUSD,hourlyTotalRevenueUSD,totalValueLockedUSD,datetime
0,1689753600,1460878.47,730.44,44233373.35,2023-07-19 08:00:00
1,1689750000,806643.11,403.32,44290156.21,2023-07-19 07:00:00
2,1689746400,1036466.04,518.23,44254886.56,2023-07-19 06:00:00
3,1689742800,656345.73,328.17,44168146.44,2023-07-19 05:00:00
4,1689739200,1637624.50,818.81,44087659.79,2023-07-19 04:00:00


In [23]:
from datetime import datetime


def timestamp_to_current_tz_datetime(ts):
    utc_date = datetime.fromtimestamp(ts)
    return utc_date.strftime("%Y-%m-%d %H:%M:%S")


df["datetime"] = df["timestamp"].apply(timestamp_to_current_tz_datetime)
df.tail()

,timestamp,hourlyVolumeUSD,hourlyTotalRevenueUSD,totalValueLockedUSD,o,c,h,l,t,datetime
1587,1684008000,2662455.28,1331.23,61592990.58,1793.70,1803.99,1805.90,1793.31,1684008000000,2023-05-14 04:00:00
1588,1684004400,5116826.36,2558.41,61377445.74,1790.12,1793.71,1801.65,1788.18,1684004400000,2023-05-14 03:00:00
1589,1684000800,6298006.55,3149.00,61259722.44,1793.99,1790.22,1808.40,1786.77,1684000800000,2023-05-14 02:00:00
1590,1683997200,2144822.87,1072.41,61175729.32,1794.92,1794.18,1799.50,1791.08,1683997200000,2023-05-14 01:00:00
1591,1683993600,2045304.59,1022.65,61190880.56,1796.19,1794.98,1797.30,1791.70,1683993600000,2023-05-14 00:00:00


In [24]:
df = df[["datetime", "h", "l", "hourlyTotalRevenueUSD", "totalValueLockedUSD"]]
df["interval"] = df["h"] - df["l"]
df.sort_values(by=["datetime"], inplace=True)
df.head()

,datetime,h,l,hourlyTotalRevenueUSD,totalValueLockedUSD,interval
1591,2023-05-14 00:00:00,1797.30,1791.70,1022.65,61190880.56,5.60
1590,2023-05-14 01:00:00,1799.50,1791.08,1072.41,61175729.32,8.42
1589,2023-05-14 02:00:00,1808.40,1786.77,3149.00,61259722.44,21.63
1588,2023-05-14 03:00:00,1801.65,1788.18,2558.41,61377445.74,13.47
1587,2023-05-14 04:00:00,1805.90,1793.31,1331.23,61592990.58,12.59


In [25]:
df.describe()

,h,l,hourlyTotalRevenueUSD,totalValueLockedUSD,interval
count,1592.00,1592.00,1592.00,1592.00,1592.00
mean,1852.10,1840.77,1833.37,52566694.77,11.33
std,69.83,69.73,2092.42,8176011.53,8.23
min,1641.40,1620.06,63.42,41703852.56,2.01
25%,1815.57,1806.49,658.20,44472744.53,6.46
50%,1863.85,1852.99,1211.43,48822178.50,8.85
75%,1901.60,1888.38,2132.02,60612443.02,13.28
max,2031.40,2011.75,26542.72,67373915.40,100.30


In [38]:
liquidity_df = pd.read_csv("../../data/arb_pool_liquidity.csv")
liquidity_records = liquidity_df.to_dict("records")
liquidity_dict = dict()

for item in liquidity_records:
    day = item["datetime"].split(" ")[0]
    if day in liquidity_dict:
        liquidity_dict[day].append(item)
    else:
        liquidity_dict[day] = [item]

In [49]:
def get_interval_liquidity(day, low_price, high_price):
    if day not in liquidity_dict:
        return 0, 0

    eth_amount = 0
    usdc_amount = 0
    records = liquidity_dict[day]
    for record in records:
        if (low_price < record["bottom_price"] < high_price) or (low_price < record["top_price"] < high_price):
            eth_amount += record["eth_amount"]
            usdc_amount += record["usdc_amount"]
    return eth_amount, usdc_amount


hourly_data = df.to_dict("records")
for item in hourly_data:
    day = item["datetime"].split(" ")[0]
    low_price = item["l"]
    high_price = item["h"]
    eth_amount, usdc_amount = get_interval_liquidity(day, low_price, high_price)
    item["interval_eth_amount"] = eth_amount
    item["interval_usdc_amount"] = usdc_amount

hourly_df = pd.DataFrame(hourly_data)
hourly_df.head()

,datetime,h,l,hourlyTotalRevenueUSD,totalValueLockedUSD,interval,interval_eth_amount,interval_usdc_amount
0,2023-05-14 00:00:00,1797.30,1791.70,1022.65,61190880.56,5.60,216.32,855797.86
1,2023-05-14 01:00:00,1799.50,1791.08,1072.41,61175729.32,8.42,372.30,855797.86
2,2023-05-14 02:00:00,1808.40,1786.77,3149.00,61259722.44,21.63,1127.49,1316397.79
3,2023-05-14 03:00:00,1801.65,1788.18,2558.41,61377445.74,13.47,525.72,1316397.79
4,2023-05-14 04:00:00,1805.90,1793.31,1331.23,61592990.58,12.59,831.91,611992.74


In [50]:
print(hourly_df.shape)
hourly_df["interval_usd"] = (hourly_df["h"] + hourly_df["l"]) / 2 * hourly_df["interval_eth_amount"] + hourly_df["interval_usdc_amount"]
hourly_df = hourly_df[hourly_df["usd"] > 0]
print(hourly_df.shape)

(1592, 8)
(1584, 9)


In [51]:
hourly_df["revenue_1k"] = hourly_df["hourlyTotalRevenueUSD"] / hourly_df["usd"] * 1000
hourly_df.head()

,datetime,h,l,hourlyTotalRevenueUSD,totalValueLockedUSD,interval,interval_eth_amount,interval_usdc_amount,usd,revenue_1k
0,2023-05-14 00:00:00,1797.30,1791.70,1022.65,61190880.56,5.60,216.32,855797.86,1243978.35,0.82
1,2023-05-14 01:00:00,1799.50,1791.08,1072.41,61175729.32,8.42,372.30,855797.86,1524188.99,0.70
2,2023-05-14 02:00:00,1808.40,1786.77,3149.00,61259722.44,21.63,1127.49,1316397.79,3343153.12,0.94
3,2023-05-14 03:00:00,1801.65,1788.18,2558.41,61377445.74,13.47,525.72,1316397.79,2260027.14,1.13
4,2023-05-14 04:00:00,1805.90,1793.31,1331.23,61592990.58,12.59,831.91,611992.74,2109108.79,0.63


In [53]:
hourly_df.sort_values(by=["revenue_1k"], ascending=False).head(100)

,datetime,h,l,hourlyTotalRevenueUSD,totalValueLockedUSD,interval,interval_eth_amount,interval_usdc_amount,usd,revenue_1k
770,2023-06-15 02:00:00,1748.10,1719.42,10614.72,48063501.82,28.68,1168.67,0.00,2026196.25,5.24
141,2023-05-19 21:00:00,1820.30,1812.81,7648.52,58253024.90,7.49,999.78,0.00,1816159.54,4.21
663,2023-06-10 15:00:00,1746.60,1725.58,5263.14,58281201.19,21.02,0.00,1373804.49,1373804.49,3.83
935,2023-06-21 23:00:00,1858.40,1840.66,7249.48,44226344.42,17.74,1056.95,0.00,1954853.89,3.71
552,2023-06-06 00:00:00,1823.50,1777.00,26542.72,58247098.89,46.50,1237.89,5111343.60,7339855.43,3.62
...,...,...,...,...,...,...,...,...,...,...
1580,2023-07-18 20:00:00,1898.20,1885.11,2920.18,44010675.22,13.09,0.00,1518341.76,1518341.76,1.92
794,2023-06-16 02:00:00,1669.70,1644.91,3857.15,44011010.35,24.79,358.03,1417771.26,2011142.96,1.92
955,2023-06-22 19:00:00,1907.80,1881.50,6264.88,43890907.56,26.30,1151.27,1097510.64,3278761.69,1.91
946,2023-06-22 10:00:00,1934.00,1906.94,4444.90,44806865.23,27.06,1216.81,0.00,2336842.49,1.90


In [55]:
# daily stats
hourly_df["day"] = hourly_df["datetime"].apply(lambda x: x.split(" ")[0])
daily_df = hourly_df.groupby(by=["day"]).agg({"h": "max", "l": "min", "hourlyTotalRevenueUSD": "sum"}).reset_index()
daily_df.head()

,day,h,l,hourlyTotalRevenueUSD
0,2023-05-14,1827.20,1786.77,33019.58
1,2023-05-15,1837.90,1785.52,53995.14
2,2023-05-16,1849.00,1796.63,50724.41
3,2023-05-17,1838.70,1782.95,43498.83
4,2023-05-18,1837.00,1794.29,48218.16


In [56]:
daily_df.rename(columns={"h": "high", "l": "low", "hourlyTotalRevenueUSD": "daily_revenue"}, inplace=True)

daily_data = daily_df.to_dict("records")
for item in daily_data:
    day = item["day"]
    low_price = item["low"]
    high_price = item["high"]
    eth_amount, usdc_amount = get_interval_liquidity(day, low_price, high_price)
    item["interval_eth_amount"] = eth_amount
    item["interval_usdc_amount"] = usdc_amount
    item["interval_usd"] = (low_price + high_price) / 2 * eth_amount + usdc_amount

daily_df = pd.DataFrame(daily_data)
daily_df.head()

,day,high,low,daily_revenue,interval_eth_amount,interval_usdc_amount,interval_usd
0,2023-05-14,1827.20,1786.77,33019.58,2690.60,1316397.79,6178269.10
1,2023-05-15,1837.90,1785.52,53995.14,3481.22,2310006.13,8616971.74
2,2023-05-16,1849.00,1796.63,50724.41,3351.68,3148868.97,9258366.65
3,2023-05-17,1838.70,1782.95,43498.83,1400.88,7103807.12,9640547.86
4,2023-05-18,1837.00,1794.29,48218.16,1483.32,5278378.29,7971569.55


In [57]:
daily_df["interval"] = daily_df["high"] - daily_df["low"]
daily_df["revenue_1k"] = daily_df["daily_revenue"] / daily_df["interval_usd"] * 1000
daily_df.sort_values(by=["revenue_1k"], ascending=False).head(100)

,day,high,low,daily_revenue,interval_eth_amount,interval_usdc_amount,interval_usd,interval,revenue_1k
30,2023-06-13,1768.40,1719.74,42850.73,1378.32,1939039.83,4342930.06,48.66,9.87
43,2023-06-26,1909.80,1865.70,54809.50,792.89,4100633.08,5597410.99,44.10,9.79
39,2023-06-22,1934.00,1846.47,91072.22,2277.39,5402234.13,9707037.55,87.53,9.38
33,2023-06-16,1688.40,1631.67,36288.06,1154.02,2228478.98,4144194.40,56.73,8.76
51,2023-07-04,1976.10,1946.77,35264.62,1414.18,1283887.48,4057712.17,29.33,8.69
...,...,...,...,...,...,...,...,...,...
22,2023-06-05,1913.27,1818.06,49800.45,2585.86,10671665.85,15496006.70,95.21,3.21
7,2023-05-21,1830.30,1802.44,22603.78,1585.17,4249323.16,7128572.94,27.86,3.17
14,2023-05-28,1861.50,1817.71,31757.61,3404.63,3918607.61,10181785.48,43.79,3.12
60,2023-07-13,1945.70,1862.44,37791.08,5547.52,1592883.72,12155749.17,83.26,3.11


In [59]:
daily_df.sort_values(by=["interval"], ascending=True).head(100)

,day,high,low,daily_revenue,interval_eth_amount,interval_usdc_amount,interval_usd,interval,revenue_1k
6,2023-05-20,1824.60,1807.26,16194.46,1691.90,1340481.06,4412844.49,17.34,3.67
55,2023-07-08,1876.30,1855.00,15078.85,513.10,1947090.78,2904352.06,21.30,5.19
31,2023-06-14,1751.70,1728.21,15116.90,705.67,1076072.17,2303908.13,23.49,6.56
35,2023-06-18,1743.60,1719.68,14058.31,846.55,708733.14,2174651.08,23.92,6.46
13,2023-05-27,1839.50,1812.92,22746.29,1502.65,3654163.56,6398310.35,26.58,3.56
...,...,...,...,...,...,...,...,...,...
48,2023-07-01,1949.00,1848.44,61860.01,903.55,9460422.70,11176010.20,100.56,5.54
62,2023-07-15,2003.10,1898.88,44754.90,4032.62,4588274.40,12455871.98,104.22,3.59
32,2023-06-15,1748.10,1620.06,65821.62,3608.71,1497367.56,7574719.85,128.04,8.69
27,2023-06-10,1846.40,1716.24,65690.92,692.35,13906552.85,15139844.76,130.16,4.34


In [61]:
hourly_df.sort_values(by=["interval"], ascending=True).head(100)

,datetime,h,l,hourlyTotalRevenueUSD,totalValueLockedUSD,interval,interval_eth_amount,interval_usdc_amount,usd,revenue_1k,day
496,2023-06-03 16:00:00,1904.48,1902.47,130.91,59781744.35,2.01,0.00,687066.73,687066.73,0.19,2023-06-03
162,2023-05-20 18:00:00,1815.80,1813.34,322.02,58075638.28,2.46,628.22,166129.68,1306073.04,0.25,2023-05-20
189,2023-05-21 21:00:00,1815.20,1812.71,124.21,61044409.78,2.49,0.00,928210.51,928210.51,0.13,2023-05-21
515,2023-06-04 11:00:00,1895.55,1893.01,262.92,59501704.92,2.54,520.30,108963.43,1094556.93,0.24,2023-06-04
493,2023-06-03 13:00:00,1903.25,1900.70,188.60,59787032.52,2.55,0.00,1030345.27,1030345.27,0.18,2023-06-03
...,...,...,...,...,...,...,...,...,...,...,...
566,2023-06-06 14:00:00,1817.79,1813.23,453.90,61094722.56,4.56,682.29,0.00,1238709.95,0.37,2023-06-06
1380,2023-07-10 12:00:00,1864.30,1859.72,439.87,47636323.96,4.58,289.46,484711.70,1023688.75,0.43,2023-07-10
195,2023-05-22 03:00:00,1810.91,1806.33,668.86,61115197.38,4.58,752.54,0.00,1361052.38,0.49,2023-05-22
153,2023-05-20 09:00:00,1813.00,1808.41,329.18,57959673.29,4.59,137.48,956533.99,1205478.05,0.27,2023-05-20


In [62]:
hourly_df.sort_values(by=["interval"], ascending=False).head(100)

,datetime,h,l,hourlyTotalRevenueUSD,totalValueLockedUSD,interval,interval_eth_amount,interval_usdc_amount,usd,revenue_1k,day
772,2023-06-15 04:00:00,1729.70,1629.40,18810.39,44905447.02,100.30,2876.74,1177120.50,6008743.28,3.13,2023-06-15
1149,2023-06-30 21:00:00,1911.60,1823.20,21929.38,46302911.35,88.40,3931.24,3554288.74,10895495.09,2.01,2023-06-30
1463,2023-07-13 23:00:00,1945.70,1881.31,15093.66,47355306.65,64.39,4855.41,0.00,9290849.21,1.62,2023-07-13
1152,2023-07-01 00:00:00,1906.30,1848.44,9536.24,46676425.51,57.86,0.00,6544775.12,6544775.12,1.46,2023-07-01
936,2023-06-22 00:00:00,1903.40,1846.47,15103.48,44210378.18,56.93,969.87,5402234.13,7220668.34,2.09,2023-06-22
...,...,...,...,...,...,...,...,...,...,...,...
573,2023-06-06 21:00:00,1820.62,1796.35,6470.56,61310537.37,24.27,912.88,2761829.46,4412758.16,1.47,2023-06-06
118,2023-05-18 22:00:00,1826.50,1802.26,6228.97,67248237.16,24.24,613.93,4087650.36,5201550.68,1.20,2023-05-18
1303,2023-07-07 07:00:00,1869.90,1845.70,5808.64,49841773.65,24.20,1515.43,240770.86,3056128.36,1.90,2023-07-07
440,2023-06-01 08:00:00,1892.60,1868.46,6151.74,59154632.76,24.14,1740.48,1370480.01,4643503.36,1.32,2023-06-01


In [64]:
hourly_df[(hourly_df["datetime"] > "2023-06-03") & (hourly_df["datetime"] < "2023-06-04")].head(25)

,datetime,h,l,hourlyTotalRevenueUSD,totalValueLockedUSD,interval,interval_eth_amount,interval_usdc_amount,usd,revenue_1k,day
480,2023-06-03 00:00:00,1895.00,1888.10,1666.45,60417486.54,6.90,0.00,1361350.25,1361350.25,1.22,2023-06-03
481,2023-06-03 01:00:00,1894.38,1887.34,723.40,60464154.97,7.04,0.00,1723098.83,1723098.83,0.42,2023-06-03
482,2023-06-03 02:00:00,1914.75,1890.94,4517.04,60667797.15,23.81,764.83,3065361.77,4520723.85,1.00,2023-06-03
483,2023-06-03 03:00:00,1911.00,1905.61,2337.68,59959732.23,5.39,474.03,360849.43,1265449.93,1.85,2023-06-03
484,2023-06-03 04:00:00,1911.10,1904.61,964.67,60293553.15,6.49,474.03,360849.43,1265236.61,0.76,2023-06-03
485,2023-06-03 05:00:00,1907.19,1902.87,758.24,60236330.27,4.32,161.53,702109.69,1009822.14,0.75,2023-06-03
486,2023-06-03 06:00:00,1909.20,1904.46,488.98,60162989.10,4.74,319.87,702109.69,1312049.31,0.37,2023-06-03
487,2023-06-03 07:00:00,1908.34,1905.01,392.52,59888129.23,3.33,161.53,360849.43,668827.59,0.59,2023-06-03
488,2023-06-03 08:00:00,1909.90,1903.02,1106.58,59872751.84,6.88,319.87,702109.69,1311930.96,0.84,2023-06-03
489,2023-06-03 09:00:00,1906.60,1898.06,1271.08,59827502.04,8.54,161.53,1726239.37,2033515.70,0.63,2023-06-03


In [65]:
liquidity_df.head()

,datetime,bottom_price,top_price,eth_amount,usdc_amount
0,2023-05-14 00:00:56,1600.44,1602.04,0.00,105267.13
1,2023-05-14 00:00:56,1602.04,1603.64,0.00,105319.54
2,2023-05-14 00:00:56,1603.64,1605.25,0.00,105381.04
3,2023-05-14 00:00:56,1605.25,1606.85,0.00,105435.96
4,2023-05-14 00:00:56,1606.85,1608.46,0.00,105493.51


In [66]:
# 回测所有区间，看最大收益所在的区间，可能需要做计算量优化
unique_price_list = liquidity_df.bottom_price.unique().tolist()
print(len(unique_price_list))

247


In [67]:
print(max(unique_price_list))
print(min(unique_price_list))

2046.77660994
1600.43980475


In [70]:
price_min = daily_df.low.min()
price_max = daily_df.high.max()
filtered_price_list = []
for liquidity_price in sorted(unique_price_list):
    if liquidity_price < price_min:
        continue
    if liquidity_price > price_max:
        continue
    filtered_price_list.append(liquidity_price)
print(len(filtered_price_list))

226


In [72]:
print(filtered_price_list[0])
print(filtered_price_list[-1])

1621.38029348
2030.46853174


In [74]:
liquidity_dict["2023-05-14"][0]

{'datetime': '2023-05-14 00:00:56',
 'bottom_price': 1600.43980475,
 'top_price': 1602.04096494,
 'eth_amount': 0.0,
 'usdc_amount': 105267.1299}

In [76]:
# 记录每天的区间收益
def convert_interval_amount():
    liquidity_interval_dict = dict()
    for day, item_list in liquidity_dict.items():
        liquidity_interval_dict[day] = dict()
        for item in item_list:
            liquidity_interval_dict[day][(item["bottom_price"], item["top_price"])] = (item["eth_amount"], item["usdc_amount"])

    interval_amount_dict = dict()
    for day in liquidity_interval_dict.keys():
        interval_amount_dict[day] = dict()
        for i in range(len(filtered_price_list) - 1):
            interval_low = filtered_price_list[i]
            last_pair_result = (0, 0)
            for j in range(i + 1, len(filtered_price_list)):
                interval_high = filtered_price_list[j]
                if (interval_low, interval_high) in liquidity_interval_dict[day]:
                    interval_amount_dict[day][(interval_low, interval_high)] = liquidity_interval_dict[day][(interval_low, interval_high)]
                else:
                    to_add_result = liquidity_interval_dict[day][(filtered_price_list[j-1], interval_high)]
                    interval_amount_dict[day][(interval_low, interval_high)] = tuple(map(sum, zip(last_pair_result, to_add_result)))
                last_pair_result = interval_amount_dict[day][(interval_low, interval_high)]

    return interval_amount_dict


interval_amount_dict = convert_interval_amount()
print(len(interval_amount_dict))

67


In [92]:
# check result
print(list(interval_amount_dict["2023-05-14"].keys())[:2])
print(interval_amount_dict["2023-05-14"][(1621.38029348, 1623.00240359)])
print(interval_amount_dict["2023-05-14"][(1621.38029348, 1624.62613654)])
print(interval_amount_dict["2023-05-14"][(1623.00240359, 1624.62613654)])

[(1621.38029348, 1623.00240359), (1621.38029348, 1624.62613654)]
(0.0, 125084.6739)
(0.0, 250235.5431)
(0.0, 125150.8692)


In [84]:
hourly_df.head()

,datetime,h,l,hourlyTotalRevenueUSD,totalValueLockedUSD,interval,interval_eth_amount,interval_usdc_amount,usd,revenue_1k,day
0,2023-05-14 00:00:00,1797.30,1791.70,1022.65,61190880.56,5.60,216.32,855797.86,1243978.35,0.82,2023-05-14
1,2023-05-14 01:00:00,1799.50,1791.08,1072.41,61175729.32,8.42,372.30,855797.86,1524188.99,0.70,2023-05-14
2,2023-05-14 02:00:00,1808.40,1786.77,3149.00,61259722.44,21.63,1127.49,1316397.79,3343153.12,0.94,2023-05-14
3,2023-05-14 03:00:00,1801.65,1788.18,2558.41,61377445.74,13.47,525.72,1316397.79,2260027.14,1.13,2023-05-14
4,2023-05-14 04:00:00,1805.90,1793.31,1331.23,61592990.58,12.59,831.91,611992.74,2109108.79,0.63,2023-05-14


In [89]:
def get_intersection_rate(hour_high, hour_low, interval_high, interval_low):
    if hour_high < interval_low or interval_high < hour_low:
        return 0
    else:
        return (min(hour_high, interval_high) - max(hour_low, interval_low)) / (hour_high - hour_low)

get_intersection_rate(1640, 1620, 1622, 1618)

0.1

In [91]:
def get_hourly_dict():
    hourly_records = hourly_df[["day", "h", "l", "hourlyTotalRevenueUSD"]].to_dict("records")
    hourly_dict = dict()
    for item in hourly_records:
        day = item["day"]
        if day in hourly_dict:
            hourly_dict[day].append((item["h"], item["l"], item["hourlyTotalRevenueUSD"]))
        else:
            hourly_dict[day] = [(item["h"], item["l"], item["hourlyTotalRevenueUSD"])]
    return hourly_dict


hourly_dict = get_hourly_dict()
print(len(hourly_dict))

66


In [93]:
def calculate_daily_interval_revenue(day, interval):
    revenue = 0

    interval_low = interval[0]
    interval_high = interval[1]
    for item in hourly_dict[day]:
        hourly_high = item[0]
        hourly_low = item[1]
        inter_rate = get_intersection_rate(hourly_high, hourly_low, interval_high, interval_low)

        hourly_revenue = item[2]
        revenue += hourly_revenue * inter_rate
    return revenue

In [94]:
# 假设在小时交易数据中高点和低点之间获得的交易利润是均匀的，假设较强，存在较大误差的可能性（获取 swap 数据后可以完全还原交易区间问题，但仍然存在流动性分布的误差）
def main():
    result = []
    for day, interval_dict in interval_amount_dict.items():
        if day not in hourly_dict:
            continue
        for interval_pair, amount_pair in interval_dict.items():
            interval_revenue = calculate_daily_interval_revenue(day, interval_pair)
            item = dict()
            item["day"] = day
            item["interval_low"] = interval_pair[0]
            item["interval_high"] = interval_pair[1]
            item["eth_amount"] = amount_pair[0]
            item["usdc_amount"] = amount_pair[1]
            item["daily_revenue"] = interval_revenue
            result.append(item)
    return result


daily_stats = main()
df = pd.DataFrame(daily_stats)
print(df.shape)

(1678050, 6)


In [96]:
df.sort_values(by="daily_revenue", ascending=False).head(20)

,day,interval_low,interval_high,eth_amount,usdc_amount,daily_revenue
1003769,2023-06-22,1725.08,1976.38,3781.71,15865309.75,91072.22
1004254,2023-06-22,1730.26,1980.34,3912.81,15616580.44,91072.22
1004252,2023-06-22,1730.26,1976.38,3781.71,15616580.44,91072.22
1004251,2023-06-22,1730.26,1974.41,3716.26,15616580.44,91072.22
1004250,2023-06-22,1730.26,1972.43,3650.73,15616580.44,91072.22
1004249,2023-06-22,1730.26,1970.46,3585.18,15616580.44,91072.22
1004248,2023-06-22,1730.26,1968.49,3519.71,15616580.44,91072.22
1004247,2023-06-22,1730.26,1966.53,3454.10,15616580.44,91072.22
1004246,2023-06-22,1730.26,1964.56,3388.23,15616580.44,91072.22
1004245,2023-06-22,1730.26,1962.60,3322.32,15616580.44,91072.22


In [97]:
daily_df[daily_df["day"] == "2023-06-22"]

,day,high,low,daily_revenue,interval_eth_amount,interval_usdc_amount,interval_usd,interval,revenue_1k
39,2023-06-22,1934.00,1846.47,91072.22,2277.39,5402234.13,9707037.55,87.53,9.38


In [98]:
df["usd"] = (df["interval_low"] + df["interval_high"]) / 2 * df["eth_amount"] + df["usdc_amount"]
df["revenue_1k"] = (df["daily_revenue"] * 1000) / df["usd"]
df.sort_values(by="revenue_1k", ascending=False).head(20)

,day,interval_low,interval_high,eth_amount,usdc_amount,daily_revenue,usd,revenue_1k
815805,2023-06-15,1637.67,1639.31,0.00,72706.72,2038.15,72706.72,28.03
815372,2023-06-15,1634.40,1636.04,0.00,71363.62,1981.27,71363.62,27.76
815374,2023-06-15,1634.40,1639.31,0.00,216630.31,6002.68,216630.31,27.71
815590,2023-06-15,1636.04,1639.31,0.00,145266.69,4021.40,145266.69,27.68
815373,2023-06-15,1634.40,1637.67,0.00,143923.59,3964.53,143923.59,27.55
815589,2023-06-15,1636.04,1637.67,0.00,72559.96,1983.25,72559.96,27.33
815157,2023-06-15,1632.77,1639.31,0.00,285452.52,7789.94,285452.52,27.29
815156,2023-06-15,1632.77,1637.67,0.00,212745.80,5751.80,212745.80,27.04
815375,2023-06-15,1634.40,1640.95,0.00,317368.56,8544.64,317368.56,26.92
815155,2023-06-15,1632.77,1636.04,0.00,140185.83,3768.54,140185.83,26.88


In [99]:
df.to_csv("../../data/daily_interval_revenue.csv", index=False)

In [100]:
daily_df[daily_df["day"] == "2023-06-15"]

,day,high,low,daily_revenue,interval_eth_amount,interval_usdc_amount,interval_usd,interval,revenue_1k
32,2023-06-15,1748.10,1620.06,65821.62,3608.71,1497367.56,7574719.85,128.04,8.69


In [101]:
hourly_df[(hourly_df["datetime"] > "2023-06-15") & (hourly_df["datetime"] < "2023-06-16")].head(50)

,datetime,h,l,hourlyTotalRevenueUSD,totalValueLockedUSD,interval,interval_eth_amount,interval_usdc_amount,usd,revenue_1k,day
768,2023-06-15 00:00:00,1744.10,1737.61,742.35,48590842.87,6.49,265.68,0.00,462513.32,1.61,2023-06-15
769,2023-06-15 01:00:00,1745.70,1730.64,2326.59,48634145.91,15.06,597.41,0.00,1038402.57,2.24,2023-06-15
770,2023-06-15 02:00:00,1748.10,1719.42,10614.72,48063501.82,28.68,1168.67,0.00,2026196.25,5.24,2023-06-15
771,2023-06-15 03:00:00,1740.70,1726.36,2893.33,47889302.14,14.34,653.99,0.00,1133718.91,2.55,2023-06-15
772,2023-06-15 04:00:00,1729.70,1629.40,18810.39,44905447.02,100.30,2876.74,1177120.50,6008743.28,3.13,2023-06-15
773,2023-06-15 05:00:00,1656.90,1630.67,4632.51,45321933.48,26.23,311.16,1110765.54,1622247.81,2.86,2023-06-15
774,2023-06-15 06:00:00,1656.30,1646.86,1532.24,44532878.12,9.44,311.16,274787.18,788694.96,1.94,2023-06-15
775,2023-06-15 07:00:00,1654.20,1645.44,1372.92,44629284.45,8.76,250.59,381488.35,794923.34,1.73,2023-06-15
776,2023-06-15 08:00:00,1656.70,1648.51,984.91,43931560.89,8.19,311.16,162530.18,676756.90,1.46,2023-06-15
777,2023-06-15 09:00:00,1656.70,1643.10,871.83,44159458.06,13.60,311.16,484979.82,998364.85,0.87,2023-06-15


In [105]:
stats_df = df.groupby(["interval_low", "interval_high"]).agg({"revenue_1k": "sum"}).reset_index()
stats_df.shape

(25425, 3)

In [115]:
stats_df["interval"] = stats_df["interval_high"] - stats_df["interval_low"]
stats_df.sort_values(by="revenue_1k", ascending=False).head(20)

,interval_low,interval_high,revenue_1k,interval
22724,1887.53,1889.42,162.61,1.89
22725,1887.53,1891.31,161.48,3.78
22651,1885.65,1889.42,161.19,3.77
22726,1887.53,1893.20,161.07,5.67
22652,1885.65,1891.31,160.96,5.67
22653,1885.65,1893.20,160.80,7.56
22797,1889.42,1891.31,160.59,1.89
22798,1889.42,1893.20,160.29,3.78
22650,1885.65,1887.53,160.15,1.89
22869,1891.31,1893.20,159.79,1.89


In [117]:
stats_df[stats_df["interval"] > 100].sort_values(by="revenue_1k", ascending=False).head(20)

,interval_low,interval_high,revenue_1k,interval
18923,1811.71,1912.23,132.30,100.52
19037,1813.52,1914.14,132.30,100.62
18924,1811.71,1914.14,132.27,102.43
19038,1813.52,1916.06,132.22,102.53
18925,1811.71,1916.06,132.19,104.35
18809,1809.90,1912.23,132.03,102.33
19039,1813.52,1917.97,132.02,104.45
18926,1811.71,1917.97,132.01,106.26
18810,1809.90,1914.14,132.01,104.24
19150,1815.34,1916.06,131.98,100.72


In [120]:
stats_df[(stats_df["interval_low"] > 1900) & (stats_df["interval"] > 128)].sort_values(by="revenue_1k", ascending=False).head(20)

,interval_low,interval_high,revenue_1k,interval
23279,1900.79,2030.47,52.61,129.68


In [108]:
df[(df["interval_low"] == 1887.53212146) & (df['interval_high'] == 1889.42050320)].head(100)

,day,interval_low,interval_high,eth_amount,usdc_amount,daily_revenue,usd,revenue_1k
22724,2023-05-14,1887.53,1889.42,115.18,0.00,0.00,217516.97,0.00
48149,2023-05-15,1887.53,1889.42,117.66,0.00,0.00,222192.27,0.00
73574,2023-05-16,1887.53,1889.42,118.36,0.00,0.00,223520.62,0.00
98999,2023-05-17,1887.53,1889.42,120.27,0.00,0.00,227130.45,0.00
124424,2023-05-18,1887.53,1889.42,122.39,0.00,0.00,231136.47,0.00
...,...,...,...,...,...,...,...,...
1573649,2023-07-14,1887.53,1889.42,0.00,226450.01,0.00,226450.01,0.00
1599074,2023-07-15,1887.53,1889.42,0.00,176813.05,0.00,176813.05,0.00
1624499,2023-07-16,1887.53,1889.42,0.00,185509.14,0.00,185509.14,0.00
1649924,2023-07-17,1887.53,1889.42,0.00,180157.34,0.00,180157.34,0.00


In [111]:
daily_df[(daily_df["day"] >= "2023-06-22") & (daily_df["day"] <= "2023-06-30")]

,day,high,low,daily_revenue,interval_eth_amount,interval_usdc_amount,interval_usd,interval,revenue_1k
39,2023-06-22,1934.00,1846.47,91072.22,2277.39,5402234.13,9707037.55,87.53,9.38
40,2023-06-23,1924.40,1861.37,47554.91,2928.17,1603326.07,7146012.49,63.03,6.65
41,2023-06-24,1938.92,1869.03,51057.17,2711.52,2672318.17,7834982.56,69.89,6.52
42,2023-06-25,1930.90,1865.64,42618.03,3267.36,1470396.07,7672722.41,65.26,5.55
43,2023-06-26,1909.80,1865.70,54809.50,792.89,4100633.08,5597410.99,44.10,9.79
44,2023-06-27,1897.60,1836.25,60349.82,2585.01,2473762.65,7299780.02,61.35,8.27
45,2023-06-28,1913.80,1845.08,53237.33,1495.82,5732579.06,8543877.74,68.72,6.23
46,2023-06-29,1880.40,1816.00,40179.61,3236.71,1492500.00,7474595.55,64.40,5.38
47,2023-06-30,1915.50,1823.20,88771.77,4121.55,3554288.74,11258905.24,92.30,7.88


In [112]:
hourly_df[(hourly_df["datetime"] > "2023-06-26") & (hourly_df["datetime"] < "2023-06-27")].head(50)

,datetime,h,l,hourlyTotalRevenueUSD,totalValueLockedUSD,interval,interval_eth_amount,interval_usdc_amount,usd,revenue_1k,day
1032,2023-06-26 00:00:00,1895.10,1884.61,2010.58,43085369.61,10.49,0.00,1779203.63,1779203.63,1.13,2023-06-26
1033,2023-06-26 01:00:00,1895.00,1884.62,1493.72,43086441.88,10.38,0.00,1531079.44,1531079.44,0.98,2023-06-26
1034,2023-06-26 02:00:00,1896.00,1880.31,2879.75,43105745.72,15.69,0.00,2292361.08,2292361.08,1.26,2023-06-26
1035,2023-06-26 03:00:00,1896.60,1885.00,1707.67,43106511.26,11.60,0.00,1779203.63,1779203.63,0.96,2023-06-26
1036,2023-06-26 04:00:00,1903.40,1888.36,1891.93,43056339.58,15.04,447.87,1361432.94,2210549.25,0.86,2023-06-26
1037,2023-06-26 05:00:00,1898.40,1892.81,658.40,42780059.29,5.59,79.57,840492.67,991320.84,0.66,2023-06-26
1038,2023-06-26 06:00:00,1907.70,1891.91,1293.83,42834691.23,15.79,678.16,840492.67,2128862.52,0.61,2023-06-26
1039,2023-06-26 07:00:00,1903.60,1897.23,699.17,42852104.74,6.37,447.87,100671.00,951818.42,0.73,2023-06-26
1040,2023-06-26 08:00:00,1905.70,1886.99,3298.99,42767311.87,18.71,563.01,1620723.35,2688377.72,1.23,2023-06-26
1041,2023-06-26 09:00:00,1889.80,1865.70,6775.10,42841525.67,24.10,0.00,3260140.41,3260140.41,2.08,2023-06-26


In [3]:
# calculate sharp ratio using mysql, and calculate cdf
import pandas as pd

from utils.config import BASE_PATH

pd.set_option('display.float_format', lambda x: '%.6f' % x)

stats_df = pd.read_csv(BASE_PATH + "/data/mysql/arb_mysql_stats_0721.csv")
stats_df.head()

,interval_low,interval_high,revenue,revenue_days,avg_revenue_rate,std_revenue_rate,interval,sharp_ratio,revenue_rate
0,1887.530000,1889.420000,163,25,0.002464,0.004070,1.890000,0.580000,89.929688
1,1885.650000,1889.420000,161,25,0.002442,0.003998,3.770000,0.590000,89.144490
2,1885.650000,1891.310000,161,26,0.002439,0.003980,5.670000,0.590000,89.015448
3,1885.650000,1893.200000,161,26,0.002436,0.003975,7.560000,0.590000,88.925854
4,1887.530000,1891.310000,161,26,0.002447,0.004035,3.780000,0.580000,89.303600


In [5]:
stats_df[(stats_df["interval_low"] > 1790) & (stats_df["interval_low"] < 1791) & (stats_df["interval_high"] > 2012) & (stats_df["interval_high"] < 2013)]

,interval_low,interval_high,revenue,revenue_days,avg_revenue_rate,std_revenue_rate,interval,sharp_ratio,revenue_rate
7459,1790.100000,2012.280000,101,56,0.001535,0.001005,222.170000,1.430000,56.040348


In [14]:
import scipy.stats as stats

avg_year = 365 * 0.001535
sigma_year = (365 ** 0.5) * 0.001005
print('The probability of increasing less than 50% in 365 days is ', stats.norm.cdf(0.5, avg_year, sigma_year))
print(avg_year, sigma_year)
# print(stats.norm(avg_year, sigma_year).cdf(0.005))
# print('The probability of increasing less than 50% in 365 days is ', stats.norm(avg_year, sigma_year).cdf(0.5))

The probability of increasing less than 50% in 365 days is  0.0008469289812057163
0.560275 0.019200498040415515


In [139]:
import scipy.stats as stats

def CDF(x, mean, std):
    """
    Calculates the cumulative distribution function (CDF) of a value `x` with
    specified mean and standard deviation.

    Args:
      x: The value to calculate the CDF for.
      mean: The mean of the distribution.
      std: The standard deviation of the distribution.

    Returns:
      The CDF of `x`.
    """

    cdf = stats.norm(mean, std).cdf(x)
    return cdf

if __name__ == "__main__":
    mean = 0.0021955703695615133
    std = 0.0009351137040227781
    x = 50 / 36500
    cdf = CDF(x, mean, std)
    print(1 - cdf)

    mean = 0.002307072727630536
    std = 0.00193766011756278
    cdf = CDF(x, mean, std)
    print(1 - cdf)

0.811382429092051
0.6856939208333734


In [134]:
stats_df["cdf"] = 1 - stats_df.apply(lambda x: CDF(50/36500, x["avg_revenue_rate"], x["std_revenue_rate"]), axis=1)
stats_df.head()

,interval_low,interval_high,revenue,revenue_days,avg_revenue_rate,std_revenue_rate,interval,sharp_ratio,revenue_rate,cdf
0,1887.530000,1889.420000,163,25,0.002464,0.004070,1.890000,0.580000,89.929688,0.605947
1,1885.650000,1889.420000,161,25,0.002442,0.003998,3.770000,0.590000,89.144490,0.605739
2,1885.650000,1891.310000,161,26,0.002439,0.003980,5.670000,0.590000,89.015448,0.605873
3,1885.650000,1893.200000,161,26,0.002436,0.003975,7.560000,0.590000,88.925854,0.605773
4,1887.530000,1891.310000,161,26,0.002447,0.004035,3.780000,0.580000,89.303600,0.605204


In [135]:
stats_df.sort_values(by="cdf", ascending=False).head(100)

,interval_low,interval_high,revenue,revenue_days,avg_revenue_rate,std_revenue_rate,interval,sharp_ratio,revenue_rate,cdf
1132,1802.680000,1923.740000,130,54,0.001964,0.001504,121.060000,1.240000,71.686662,0.653638
1131,1802.680000,1921.810000,130,54,0.001970,0.001519,119.140000,1.230000,71.889568,0.653543
1136,1804.480000,1923.740000,130,54,0.001971,0.001523,119.260000,1.230000,71.945197,0.653509
1233,1800.870000,1923.740000,129,54,0.001956,0.001486,122.860000,1.250000,71.400266,0.653455
1135,1804.480000,1921.810000,130,54,0.001977,0.001538,117.330000,1.220000,72.156920,0.653452
...,...,...,...,...,...,...,...,...,...,...
1640,1802.680000,1937.250000,126,55,0.001906,0.001392,134.570000,1.300000,69.551440,0.649828
1154,1809.900000,1925.660000,130,54,0.001973,0.001567,115.760000,1.200000,72.015516,0.649821
890,1808.090000,1916.060000,132,54,0.001997,0.001629,107.960000,1.160000,72.878358,0.649818
932,1802.680000,1914.140000,131,53,0.001980,0.001585,111.470000,1.190000,72.261102,0.649815


In [137]:
stats_df[(stats_df["interval_low"] > 1795) & (stats_df["interval_low"] < 1796) & (stats_df["interval_high"] > 1905) & (stats_df["interval_high"] < 2007)].head(100)

,interval_low,interval_high,revenue,revenue_days,avg_revenue_rate,std_revenue_rate,interval,sharp_ratio,revenue_rate,cdf
1389,1795.480000,1906.500000,128,53,0.001934,0.001574,111.020000,1.160000,70.578905,0.639903
1390,1795.480000,1908.410000,128,53,0.001937,0.001561,112.930000,1.180000,70.712359,0.641900
1391,1795.480000,1910.320000,128,53,0.001941,0.001548,114.840000,1.190000,70.833974,0.643849
1392,1795.480000,1912.230000,128,53,0.001944,0.001535,116.750000,1.200000,70.962517,0.645821
1393,1795.480000,1914.140000,128,53,0.001945,0.001520,118.660000,1.210000,71.006088,0.647481
1394,1795.480000,1916.060000,128,54,0.001946,0.001504,120.580000,1.230000,71.021450,0.649072
1395,1795.480000,1917.970000,128,54,0.001945,0.001487,122.490000,1.240000,70.983595,0.650446
1396,1795.480000,1919.890000,128,54,0.001943,0.001471,124.410000,1.250000,70.910394,0.651499
1397,1795.480000,1921.810000,128,54,0.001938,0.001457,126.330000,1.260000,70.752847,0.651805
1398,1795.480000,1923.740000,128,54,0.001934,0.001443,128.260000,1.270000,70.582026,0.652028
